# Seq2Seq

- seq2seq : 언어를 다른언어로 해석해주는 뉴럴기계번역 모델.

    - 시퀀스를 입력받아 또 다른 시퀀스를 출력. 문장을 다른문장으로 번역해주는 모델이다.

    - 각자 다른역할을 하는 두개의 RNN을 이어붙인 모델이다.
- 학습하기 위해서는 병렬 말뭉치라는 원문과 번역문이 쌍을 이루는 형태의 많은 텍스트데이터가 필요하다. 
- 예제는 매우 간소화 한 예제, 영어문자열 "hello"를 스페인어 문자열 "hola"로 번역하는 미니 seq2seq 구현

## seq2seq 동작 설명

1. 원문을 이해하고(인코더) 번역문을 작성하는(디코더) 두가지 동작으로 (두 RNN으로)구성.
2. 인코더
    - 원문속 모든 단어를 입력받아 문장의 뜻을 내포하는 하나의 고정크기텐서를 만듬
    - 압축된 텐서는 원문의 뜻과 내용을 압축하고 있다 하여 "문맥벡터"라고한다.
    - 오토인코더는 정보를 추려서 압축, rnn은 동적인 시계열 데이터를 정적인 시계열데이터로 압축

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt

In [5]:
## 간단한 영단어 hello를 hola로 바꾸는 것이므로 단어단위의 워드임베딩이 아닌 글자단위의 캐릭터임베딩을 사용
vocab_size=256 # 데이터속에 총ㄷ 몇종류의 토큰이 있는지 정의. 영문만 다루므로 아스키코드로 대신, 아스키코드로는 총 256개의 글자표현가능
x_=list(map(ord,"hello")) #아스키코드로 변환
y_=list(map(ord,"hola"))
x=torch.LongTensor(x_) # 파이토치 텐서로 변환
y=torch.LongTensor(y_)

In [6]:
print(map(ord,"hello"))
print(list(map(ord,"hello")))

[104, 101, 108, 108, 111]


In [7]:
class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(Seq2Seq, self).__init__()
        self.n_layers = 1
        self.hidden_size = hidden_size #임베딩된 토큰의 차원값
        #원래는 인코더와 디코더의 체계가 다른경우를 대비해 따로 만들어야하는데 예제에서는 모두 아스키코드로 나타내므로 임베딩은 하나만 만든다
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.encoder = nn.GRU(hidden_size, hidden_size) #인코더와 디코더를 gru로 대체
        self.decoder = nn.GRU(hidden_size, hidden_size)
        self.project = nn.Linear(hidden_size, vocab_size) #디코더가 번역문의 다음토큰을 예상해내는 작은 신경망

    def forward(self, inputs, targets):
        # 인코더에 들어갈 입력
        initial_state = self._init_state() #첫번째 은닉벡터를 정의.
        embedding = self.embedding(inputs).unsqueeze(1)#원문의 모든 문자를 임베딩
        # embedding = [seq_len, batch_size, embedding_size]
        
        # 인코더 (Encoder)
        # 문맥벡터인 encoder_state를 만들고 이를 디코더의 첫번째 은닉벡터로 지정.
        encoder_output, encoder_state = self.encoder(embedding, initial_state)
        # encoder_output = [seq_len, batch_size, hidden_size]
        # encoder_state  = [n_layers, seq_len, hidden_size]

        # 디코더에 들어갈 입력
        decoder_state = encoder_state # 문맥벡터
        decoder_input = torch.LongTensor([0]) #아스키값으로 null문자를 뜻한 0을 넣어줌으로써 디코더에 문장의 시작을 알림
        
        # 디코더 (Decoder)
        outputs = []
        
        for i in range(targets.size()[0]):
            decoder_input = self.embedding(decoder_input).unsqueeze(1)
            decoder_output, decoder_state = self.decoder(decoder_input, decoder_state)
            projection = self.project(decoder_output) #다음 예상글자를 예측.
            outputs.append(projection)
            
            #티처 포싱(Teacher Forcing) 사용 (디코더의 출력이아닌 정답을 다음 은닉벡터에 넣어줌)
            decoder_input = torch.LongTensor([targets[i]])

        outputs = torch.stack(outputs).squeeze()
        return outputs
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_size).zero_()

In [8]:
seq2seq=Seq2Seq(vocab_size,16)
criterion=nn.CrossEntropyLoss()
optimizer=th.optim.Adam(seq2seq.parameters(),lr=1e-3)

NameError: name 'th' is not defined

In [ ]:
log[]
for i in range(1000):
    prediction=seq2seq(x,y)
    loss=criterion(prediction,y)
    optimizer.zero.grad()
    loss.backward()
    optimizer.step()
    loss_val=loss.data
    log.append(loss_val)
    if i%100 ==0:
        print("\n 반복:%d 오차 %s " % (i,loss_val.item()))
        _,top1=prediction.data.topk(1,1)
        print([chr(c) for c in top1.squeeze().numpy().tolist()])